## Lab Handling Imbalanced Data & Cross Validation 

### Continuation of Labs 7.2, 7.3 & 7.4 in previous Notebook 

### Round 4

- fit a Random forest Classifier on the data and compare the accuracy.
- tune the hyper paramters with gridsearch and check the results.


## Data Preprocessing

In [1]:
import pandas as pd 

In [2]:
churnData = pd.read_csv('DATA_Customer-Churn.csv') # import csv, create dataframe 

In [3]:
churnData # display dataframe 

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [4]:
churnData.info() # get overview of data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [5]:
# Count null values in each column of the DataFrame
null_counts = churnData.isnull().sum()

# Print the null value counts
print(null_counts)

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [6]:
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'], errors='coerce') # By adding errors='coerce', the function will convert non-numeric values to NaN, 
# allowing the conversion to proceed without raising an error.

In [7]:
# Print the null value counts
print(null_counts)

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [8]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7032 non-null   float64
 15  Churn             7043 non-null   object 
dtypes: float64(2), int64(2), object(12)
memory

In [9]:
# Filter the DataFrame to show only entirely duplicated rows
duplicate_rows = churnData[churnData.duplicated(keep=False)]

# Print the duplicated rows
print(duplicate_rows)

      gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
22      Male              0      No         No       1          Yes   
33      Male              0      No         No       1          Yes   
100     Male              0      No         No       1          Yes   
128     Male              0      No         No       1          Yes   
211   Female              0      No         No       1           No   
...      ...            ...     ...        ...     ...          ...   
6706  Female              0      No         No       1          Yes   
6764  Female              0      No         No       1          Yes   
6774  Female              0      No         No       1          Yes   
6789  Female              0      No         No       1          Yes   
6924    Male              0      No         No       1          Yes   

           OnlineSecurity         OnlineBackup     DeviceProtection  \
22    No internet service  No internet service  No internet service   
33   

In [10]:
# Drop entirely duplicated rows from the DataFrame
churnData.drop_duplicates(inplace=True)

In [11]:
# List of columns to keep
columns_to_keep = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges', 'Churn']

# Drop all columns except the ones in columns_to_keep
churnData.drop(columns=churnData.columns.difference(columns_to_keep), inplace=True)
churnData

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn
0,0,1,29.85,29.85,No
1,0,34,56.95,1889.50,No
2,0,2,53.85,108.15,Yes
3,0,45,42.30,1840.75,No
4,0,2,70.70,151.65,Yes
...,...,...,...,...,...
7038,0,24,84.80,1990.50,No
7039,0,72,103.20,7362.90,No
7040,0,11,29.60,346.45,No
7041,1,4,74.40,306.60,Yes


In [12]:
# Assuming churn is my target variable

churnData['Churn'].nunique

<bound method IndexOpsMixin.nunique of 0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 6994, dtype: object>

In [13]:
churnData['Churn']=churnData['Churn'].map({'No': 0, 'Yes': 1})

In [14]:
churnData

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn
0,0,1,29.85,29.85,0
1,0,34,56.95,1889.50,0
2,0,2,53.85,108.15,1
3,0,45,42.30,1840.75,0
4,0,2,70.70,151.65,1
...,...,...,...,...,...
7038,0,24,84.80,1990.50,0
7039,0,72,103.20,7362.90,0
7040,0,11,29.60,346.45,0
7041,1,4,74.40,306.60,1


In [15]:
churnData = churnData.dropna()

In [16]:
# Reset the index of the DataFrame
churnData.reset_index(drop=True, inplace=True)

## Split data into train and test set 

In [50]:
from sklearn.model_selection import train_test_split

X = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]
y = churnData['Churn']

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

log_model = LogisticRegression() 

## Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)


In [18]:
y_train.value_counts(normalize=False)

Churn
0    4128
1    1458
Name: count, dtype: int64

In [19]:
NEG_CLASS_CNT = 4128 # number of negative labels

In [20]:
print("The majority class (negative cases) represents {:.2f}% of the data".format(NEG_CLASS_CNT/len(y_train)*100))

The majority class (negative cases) represents 73.90% of the data


In [21]:
data = pd.concat([X_train, y_train], axis=1)

In [22]:
data.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
6163,25,0,54.75,1266.35,0
5812,17,0,88.25,1460.65,1
5340,2,0,80.25,144.55,1
941,32,1,91.35,2896.55,0
1957,20,0,19.35,433.75,0


In [23]:
data.shape

(5586, 5)

## Balancing the data - SMOTE

In [27]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [35]:
category_0 = y_train == 0 # negative class (majority)
category_1 = y_train == 1 # positive class (minority)

In [36]:
print(category_0.shape)
print(category_1.shape)

(5586,)
(5586,)


In [38]:
X_sm, y_sm = smote.fit_resample(X_train, y_train)
y_sm.value_counts()

Churn
0    4128
1    4128
Name: count, dtype: int64

In [40]:
X_sm.value_counts()

tenure  SeniorCitizen  MonthlyCharges  TotalCharges
1       0              19.950000       19.950000       5
                       69.950000       69.950000       5
                       75.300000       75.300000       5
                       70.600000       70.600000       5
                       19.300000       19.300000       5
                                                      ..
10      1              92.500000       934.100000      1
                       89.800000       914.300000      1
                       86.650000       856.650000      1
                       86.610077       856.823323      1
72      1              116.750000      8277.050000     1
Name: count, Length: 8118, dtype: int64

In [43]:
data = pd.concat([X_sm, y_sm], axis=1)

In [45]:
data.reset_index(drop=True, inplace=True)

In [46]:
data

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,25,0,54.750000,1266.350000,0
1,17,0,88.250000,1460.650000,1
2,2,0,80.250000,144.550000,1
3,32,1,91.350000,2896.550000,0
4,20,0,19.350000,433.750000,0
...,...,...,...,...,...
8251,51,0,99.570237,5026.260089,1
8252,1,0,69.640705,69.640705,1
8253,4,0,72.766328,326.666647,1
8254,11,0,62.201723,713.731273,1


In [47]:
data['Churn'].value_counts()

Churn
0    4128
1    4128
Name: count, dtype: int64

## Split the balanced data 

In [73]:
from sklearn.model_selection import train_test_split

X = data[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]
y = data['Churn']

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

log_model = LogisticRegression() 

## Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)


## Fit a RandomForest Classifier

In [87]:
from sklearn.ensemble import RandomForestClassifier
# Initialize the RandomForestClassifier
clf = RandomForestClassifier()

clf.fit(X_train, y_train)

RandomForestClassifier()

In [88]:
print("test prediction accuracy score: %.2f" %(clf.score(X_test, y_test)))

test prediction accuracy score: 0.80


## Tune the hyper paramters with gridsearch and check the results.

In [89]:
RAND_STATE = 42

# Create a new parameter grid with ranges
param_grid = {
    "max_depth": [6, 8, 10],  # Test three different values for max_depth
    "min_samples_leaf": [10, 20, 30],  # Test three different values for min_samples_leaf
    "n_estimators": [50, 100, 150],  # Test three different values for n_estimators
    "bootstrap": [True],
    "oob_score": [True],
    "random_state": [RAND_STATE]
}

# Initialize the RandomForestClassifier
clf = RandomForestClassifier()

# Initialize GridSearchCV with the classifier and parameter grid
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters:", best_params)
print("Best cross-validation score:", best_score)

Best parameters: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 10, 'n_estimators': 150, 'oob_score': True, 'random_state': 42}
Best cross-validation score: 0.757266304223155
